### Online Retails Data with KMeans Clustering
- Data Source: https://archive.ics.uci.edu/dataset/502/online+retail+ii
- date: 2024-11-01

In [2]:
### Setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('display.max_columns', 500)

In [3]:
import warnings
warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)

### Basic Data Exploration

In [5]:
df = pd.read_excel('datasets/online_retail_II.xlsx', sheet_name='Year 2010-2011')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,"17,850.000",United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,"17,850.000",United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,"17,850.000",United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,"17,850.000",United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,"17,850.000",United Kingdom


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


<figure>
  <img src="pics/online-retail-addinfo.png" width="700">
  <figcaption>Additional Info on dataset</figcaption>
</figure>

In [7]:
# Filter for Invoices that are not 6-digit numbers
df['Invoice'] = df['Invoice'].astype("str")
non_standard_invoices = df[~df['Invoice'].str.match("^\\d{6}$")]
non_standard_invoices.head()

# Filter for unique values in the Invoice column with non-standard values
non_standard_invoices['Invoice'].str.replace("[0-9]", "", regex=True).unique()

array(['C', 'A'], dtype=object)

In [8]:
# Is all invoices that start with 'C' are negative quantities?
df[df['Invoice'].str.startswith("C")]['Quantity'].max()

-1

All invoices with C prefix are cancelled invoices. While invoices with A prefix are adjustments. We will remove these from the dataset.

In [9]:
# Update dataset excluding the non-standard invoices
df = df[df['Invoice'].str.match("^\\d{6}$")]

# Occasionally check the dataset
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,"532,619.000",532619,"532,619.000","397,925.000"
mean,10.240,2011-07-04 17:06:17.472038400,3.868,"15,294.309"
min,"-9,600.000",2010-12-01 08:26:00,0.000,"12,346.000"
25%,1.000,2011-03-28 12:13:00,1.250,"13,969.000"
50%,3.000,2011-07-20 11:54:00,2.080,"15,159.000"
75%,10.000,2011-10-19 12:21:00,4.130,"16,795.000"
max,"80,995.000",2011-12-09 12:50:00,"13,541.330","18,287.000"
std,159.594,NaN,32.470,"1,713.173"


In [10]:
# Check for negative values in the Quantity column and price is less than 0
df[df['Quantity'] < 0 ]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.000,NaN,United Kingdom
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.000,NaN,United Kingdom
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.000,NaN,United Kingdom
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.000,NaN,United Kingdom
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.000,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.000,NaN,United Kingdom
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.000,NaN,United Kingdom
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.000,NaN,United Kingdom
536910,581226,23090,missing,-338,2011-12-08 09:56:00,0.000,NaN,United Kingdom


In [11]:
df_cleaned = df[df['Quantity'] > 0 ]
df_cleaned.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,"531,283.000",531283,"531,283.000","397,925.000"
mean,10.655,2011-07-04 18:15:52.516755968,3.878,"15,294.309"
min,1.000,2010-12-01 08:26:00,0.000,"12,346.000"
25%,1.000,2011-03-28 11:59:00,1.250,"13,969.000"
50%,3.000,2011-07-20 12:01:00,2.080,"15,159.000"
75%,10.000,2011-10-19 12:35:00,4.130,"16,795.000"
max,"80,995.000",2011-12-09 12:50:00,"13,541.330","18,287.000"
std,156.831,NaN,32.511,"1,713.173"


In [12]:
# Convert to string and replace 'nan' with actual NaN (None)
df_cleaned['Customer ID'] = df_cleaned['Customer ID'].astype(str).replace('nan', None)

In [13]:
# Check for missing values
df_cleaned.isna().sum()

Invoice             0
StockCode           0
Description       592
Quantity            0
InvoiceDate         0
Price               0
Customer ID    133358
Country             0
dtype: int64

In [14]:
# First, let's see the invoices with missing Customer IDs
missing_customer_invoices = df_cleaned[df_cleaned['Customer ID'].isna()]['Invoice'].unique()

# Get count of how many times each invoice appears
invoice_analysis = df_cleaned[df_cleaned['Customer ID'].isna()].groupby('Invoice').agg({
    'Customer ID': 'count',  # Count of rows for each invoice
    'StockCode': 'count',    # Number of items in invoice
    'Price': 'sum'          # Total value of invoice
}).rename(columns={'Customer ID': 'Number_of_Items'})

# Check if these invoices ever appear with a Customer ID
invoices_with_customer = df_cleaned[
    (df_cleaned['Invoice'].isin(missing_customer_invoices)) & 
    (df_cleaned['Customer ID'].notna())
]

# Summary of findings
print(f"Number of invoices with missing Customer ID: {len(missing_customer_invoices)}")
print("\nSample of invoices with missing Customer IDs:")
print(invoice_analysis.head())

if len(invoices_with_customer) > 0:
    print("\nWARNING: Found invoices that appear with both missing and non-missing Customer IDs:")
    print(invoices_with_customer[['Invoice', 'Customer ID']].head())
else:
    print("\nAll invoices with missing Customer IDs are consistently missing (good consistency)")

# Additional analysis by invoice characteristics
print("\nAnalysis of transactions with missing Customer IDs:")
missing_analysis = df_cleaned[df_cleaned['Customer ID'].isna()].agg({
    'Invoice': 'nunique',
    'StockCode': 'count',
    'Price': ['sum', 'mean'],
    'Quantity': ['sum', 'mean']
}).round(2)
print(missing_analysis)

# Check if there are any patterns
missing_patterns = df_cleaned[df_cleaned['Customer ID'].isna()].groupby(['Country', 'Invoice']).size().reset_index()
print("\nDistribution of missing Customer IDs by country:")
print(missing_patterns.groupby('Country').size().sort_values(ascending=False))

Number of invoices with missing Customer ID: 2189

Sample of invoices with missing Customer IDs:
         Number_of_Items  StockCode     Price
Invoice                                      
536414                 0          1     0.000
536544                 0        527 2,987.720
536545                 0          1     0.000
536546                 0          1     0.000
536547                 0          1     0.000

All invoices with missing Customer IDs are consistently missing (good consistency)

Analysis of transactions with missing Customer IDs:
          Invoice   StockCode       Price    Quantity
nunique 2,189.000         NaN         NaN         NaN
count         NaN 133,358.000         NaN         NaN
sum           NaN         NaN 820,385.100 479,282.000
mean          NaN         NaN       6.150       3.590

Distribution of missing Customer IDs by country:
Country
United Kingdom    2134
EIRE                28
Hong Kong           11
Unspecified          5
France               3
I

In [15]:
# Get value counts and percentages for each country
country_distribution = df_cleaned['Country'].value_counts()
country_percentages = df_cleaned['Country'].value_counts(normalize=True) * 100

# Display both counts and percentages
country_analysis = pd.DataFrame({
    'Count': country_distribution,
    'Percentage': country_percentages
})

# Sort by count in descending order
country_analysis = country_analysis.sort_values('Count', ascending=False)

# Round percentages to 2 decimal places
country_analysis['Percentage'] = country_analysis['Percentage'].round(2)

print("Top 5 Distribution of data by country:")
print(country_analysis[:5])

Top 5 Distribution of data by country:
                 Count  Percentage
Country                           
United Kingdom  486283      91.530
Germany           9042       1.700
France            8409       1.580
EIRE              7894       1.490
Spain             2485       0.470


In [16]:
df_cleaned.describe()

,Quantity,InvoiceDate,Price
count,"531,283.000",531283,"531,283.000"
mean,10.655,2011-07-04 18:15:52.516755968,3.878
min,1.000,2010-12-01 08:26:00,0.000
25%,1.000,2011-03-28 11:59:00,1.250
50%,3.000,2011-07-20 12:01:00,2.080
75%,10.000,2011-10-19 12:35:00,4.130
max,"80,995.000",2011-12-09 12:50:00,"13,541.330"
std,156.831,NaN,32.511


### Data Preprocessing

In [17]:
# Check for negative values in Quantity and Price
negative_quantity = df[df['Quantity'] < 0]
negative_price = df[df['Price'] < 0]

# Summary of findings
print(f"Number of rows with negative Quantity: {len(negative_quantity)}")
print(f"Number of rows with negative Price: {len(negative_price)}")

# Additional analysis
print("\nSample of rows with negative Quantity:")
print(negative_quantity.head())

print("\nSample of rows with negative Price:")
print(negative_price.head())

# Check for negative values in Quantity with StockCode as 'D'
negative_d = df[(df['Quantity'] < 0) & (df['StockCode'] == 'D')]
print(f"Number of rows with negative Quantity and StockCode 'D': {len(negative_d)}")
negative_d.head()


Number of rows with negative Quantity: 1336
Number of rows with negative Price: 0

Sample of rows with negative Quantity:
     Invoice StockCode Description  Quantity         InvoiceDate  Price  \
2406  536589     21777         NaN       -10 2010-12-01 16:50:00  0.000   
4347  536764    84952C         NaN       -38 2010-12-02 14:42:00  0.000   
7188  536996     22712         NaN       -20 2010-12-03 15:30:00  0.000   
7189  536997     22028         NaN       -20 2010-12-03 15:30:00  0.000   
7190  536998     85067         NaN        -6 2010-12-03 15:30:00  0.000   

      Customer ID         Country  
2406          NaN  United Kingdom  
4347          NaN  United Kingdom  
7188          NaN  United Kingdom  
7189          NaN  United Kingdom  
7190          NaN  United Kingdom  

Sample of rows with negative Price:
Empty DataFrame
Columns: [Invoice, StockCode, Description, Quantity, InvoiceDate, Price, Customer ID, Country]
Index: []
Number of rows with negative Quantity and StockCode '

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


In [18]:
# Using only the UK data with non-missing values
df_uk = df[(df['Country'] == 'United Kingdom') & df.notna().all(axis=1)]

# Use only 2011 data
df_uk = df_uk[df_uk['InvoiceDate'].dt.year == 2011]

# New Column: Total Price
df_uk['TotalPrice'] = df_uk['Quantity'] * df_uk['Price']

print("Data types after conversion:\n")
df_uk.info()

print("\nSummary statistics for UK data:")
df_uk.describe()

Data types after conversion:

<class 'pandas.core.frame.DataFrame'>
Index: 330402 entries, 42481 to 541893
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      330402 non-null  object        
 1   StockCode    330402 non-null  object        
 2   Description  330402 non-null  object        
 3   Quantity     330402 non-null  int64         
 4   InvoiceDate  330402 non-null  datetime64[ns]
 5   Price        330402 non-null  float64       
 6   Customer ID  330402 non-null  float64       
 7   Country      330402 non-null  object        
 8   TotalPrice   330402 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 25.2+ MB

Summary statistics for UK data:


,Quantity,InvoiceDate,Price,Customer ID,TotalPrice
count,"330,402.000",330402,"330,402.000","330,402.000","330,402.000"
mean,12.112,2011-07-26 19:16:00.633228544,2.955,"15,538.358",20.610
min,1.000,2011-01-04 10:00:00,0.000,"12,346.000",0.000
25%,2.000,2011-05-04 11:50:00,1.250,"14,180.000",4.160
50%,4.000,2011-08-17 10:59:00,1.790,"15,505.000",10.200
75%,12.000,2011-10-26 15:48:00,3.750,"16,923.000",17.700
max,"80,995.000",2011-12-09 12:49:00,"8,142.750","18,287.000","168,469.600"
std,196.835,NaN,18.446,"1,591.761",337.038


In [19]:
negative_quantity = df_uk[df_uk['Quantity'] < 0]

print(f"Number of rows with negative Quantity: {len(negative_quantity)}")
negative_quantity.head()

Number of rows with negative Quantity: 0


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
